# Multilayer Perceptron (MLP) for the names dataset 

In [2]:
import torch
import torch.functional as F
import matplotlib.pyplot as plt



In [3]:
words = open('data/names.txt', 'r').read().split()
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [4]:
len(words)

32033

## Tokenization

In [5]:
chars = sorted(list(set(''.join(words))))
stringToIndex = {char:index + 1 for index, char in enumerate(chars)}
stringToIndex['.'] = 0
indexToString = {index:char for char, index in stringToIndex.items()}
print(indexToString)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [6]:
# Building the dataset
block_size = 3
X, Y = [], []
for word in words:
    # print(word)
    context = [0] * block_size
    for ch in word + '.':
        ix = stringToIndex[ch]
        X.append(context)
        Y.append(ix)
        # print(''.join(indexToString[i] for i in context), '------->', indexToString[ix])
        context = context[1:] + [ix] # Crop and append
        
X = torch.tensor(X)
Y = torch.tensor(Y)
X.shape

torch.Size([228146, 3])

In [7]:
# Embedding layer
C = torch.randn((27, 2))
emb = C[X]

# Weights to the hidden layer
W1 = torch.randn((6, 100))
b1 = torch.randn((100,))

## 3 ways to reshape the embedding layer

In [8]:
# Not useful, hard coded. What if my block_size changes?
torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], dim=1)

tensor([[-0.3582,  0.9743, -0.3582,  0.9743, -0.3582,  0.9743],
        [-0.3582,  0.9743, -0.3582,  0.9743,  0.6907, -1.6614],
        [-0.3582,  0.9743,  0.6907, -1.6614,  0.2779, -0.3458],
        ...,
        [-2.2907,  0.9125, -2.2907,  0.9125, -0.0355, -0.8173],
        [-2.2907,  0.9125, -0.0355, -0.8173, -2.2907,  0.9125],
        [-0.0355, -0.8173, -2.2907,  0.9125, -0.6835,  1.3147]])

In [9]:
# Not efficient, we are creating another tensor
torch.cat(torch.unbind(emb, dim = 1), dim = 1)

tensor([[-0.3582,  0.9743, -0.3582,  0.9743, -0.3582,  0.9743],
        [-0.3582,  0.9743, -0.3582,  0.9743,  0.6907, -1.6614],
        [-0.3582,  0.9743,  0.6907, -1.6614,  0.2779, -0.3458],
        ...,
        [-2.2907,  0.9125, -2.2907,  0.9125, -0.0355, -0.8173],
        [-2.2907,  0.9125, -0.0355, -0.8173, -2.2907,  0.9125],
        [-0.0355, -0.8173, -2.2907,  0.9125, -0.6835,  1.3147]])

In [10]:
# Efficient way
emb.view(-1, block_size * C.shape[1])

tensor([[-0.3582,  0.9743, -0.3582,  0.9743, -0.3582,  0.9743],
        [-0.3582,  0.9743, -0.3582,  0.9743,  0.6907, -1.6614],
        [-0.3582,  0.9743,  0.6907, -1.6614,  0.2779, -0.3458],
        ...,
        [-2.2907,  0.9125, -2.2907,  0.9125, -0.0355, -0.8173],
        [-2.2907,  0.9125, -0.0355, -0.8173, -2.2907,  0.9125],
        [-0.0355, -0.8173, -2.2907,  0.9125, -0.6835,  1.3147]])

In [11]:
h = torch.tanh(emb.view(-1, block_size * C.shape[1])@(W1) + b1)
h.shape

torch.Size([228146, 100])

### Output layer

In [15]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)
logits = h@W2 + b2
counts = logits.exp()
prob = counts/counts.sum(1, keepdim=True)


In [19]:
prob.sum(1)

tensor([1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000])

In [27]:
prob[torch.arange(prob.shape[0]), Y]

tensor([2.3648e-07, 1.7545e-11, 2.0323e-14,  ..., 2.5863e-14, 2.2179e-03,
        4.1952e-14])

In [26]:
prob.shape[0]

228146